# PyCity Schools Analysis

* In general, charter schools performed better than district shools in terms of math, reading, and overall scores. The top five highest-performing schools were all charter schools, while the bottom five were district schools.

* As a whole, smaller schools had higher average scores than the medium sized, while medium sized schools still out-performed large schools. However, medium schools had a higher passing percentage than small schools. Large schools seemed to be doing the worst in terms of scores.

* Average scores of math and reading were quite similar across different grades within the same schools. The level of performance was quite uniformed throughout the whole school.

* The more schools spent (per student), the worse their performances were. The schools that had the best perfermances (i.e. highest average scores in math and reading, and highest passing percentages) were the ones with the least spending per student. To see it more in details, schools that spent the least ( less than $585) had 90.369459% overall passing rate, while schools spent the most, from $645 to $680, only got 53.526855% overall passing rate.
---

In [99]:
#setting up
import pandas as pd
import os

In [100]:
school_path = os.path.join("Resources","schools_complete.csv")
student_path = os.path.join("Resources","students_complete.csv")

In [101]:
school_df = pd.read_csv(school_path, encoding="utf8")
student_df = pd.read_csv(student_path, encoding="utf8")

In [102]:
combined_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])
combined_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


 ## District Summary

In [103]:
# Total number of unique schools
school_count = school_df["school_name"].count()
school_count

15

In [104]:
# Total students
student_count = student_df["Student ID"].count()
student_count

39170

In [105]:
# Total budget
total_budget = school_df["budget"].sum()
total_budget

24649428

In [106]:
# Average math score
average_math_score = combined_df["math_score"].sum()/student_count
average_math_score

78.98537145774827

In [107]:
# Average reading score
average_reading_score = combined_df["reading_score"].sum()/student_count
average_reading_score

81.87784018381414

In [108]:
# %passing math (the percentage of students who passed math)
passing_math = combined_df[(combined_df["math_score"] >= 60)].count()["Student ID"]
percentage_pass_math = passing_math / float(student_count)*100
percentage_pass_math



92.4457492979321

In [109]:
# %passing reading (the percentage of students who passed reading)
passing_reading = combined_df[(combined_df["reading_score"] >= 60)].count()["Student ID"]
percentage_pass_reading = passing_reading / float(student_count)*100
percentage_pass_reading

100.0

In [110]:
# %overall passing (the percentage of students who passed math AND reading)
overall_pass = combined_df[(combined_df["math_score"] >= 60) & (combined_df["reading_score"] >= 60)].count()["Student ID"]
percentage_overall_pass = overall_pass / float(student_count)*100
percentage_overall_pass

92.4457492979321

In [111]:
# district summary
district_summary = pd.DataFrame([{\
                                  "Total Schools":school_count, \
                                  "Total Students":student_count,\
                                  "Total Budget":total_budget,\
                                  "Average Math Score":average_math_score,\
                                  "Average Reading Score":average_reading_score, \
                                  "% Passing Math":percentage_pass_math ,\
                                  "% Passing Reading":percentage_pass_reading,
                                 "%Overall Passing": percentage_overall_pass}])
# formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,92.445749,100.0,92.445749


## School Summary

In [112]:
# School name
unique_schools = pd.Series(combined_df["school_name"].unique())
unique_schools

0         Huang High School
1      Figueroa High School
2       Shelton High School
3     Hernandez High School
4       Griffin High School
5        Wilson High School
6       Cabrera High School
7        Bailey High School
8        Holden High School
9          Pena High School
10       Wright High School
11    Rodriguez High School
12      Johnson High School
13         Ford High School
14       Thomas High School
dtype: object

In [123]:
  # School type  # Total school budget
school_summary_df = school_df.set_index(["school_name"])
school_summary_df = school_summary_df.rename(columns={"type": "School Type", "size" : "Total Students" , "budget": 
                                                     "Total School Budget"})
school_summary_df = school_summary_df.drop(columns = "School ID")

In [124]:
# Per student budget
per_student_budget = school_summary_df["Total School Budget"] / school_summary_df["Total Students"]
school_summary_df["Per Student Budget"]= per_student_budget

school_summary_df.loc[:, "Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,.2f}".format)
school_summary_df.loc[:, "Total School Budget"] =school_summary_df["Total School Budget"].map("${:,.2f}".format)

In [125]:
# Average math score
per_school_math = combined_df.groupby(["school_name"]).mean()["math_score"]
school_summary_df["Average Math Score"] = per_school_math

In [126]:
# Average reading score
per_school_reading = combined_df.groupby(["school_name"]).mean()["reading_score"]
school_summary_df["Average Reading Score"] = per_school_reading

In [127]:
################################
# to get the number of passing scores
passing_combined_df = combined_df.rename(columns={"math_score":"math_score_pass",
                                                 "reading_score":"reading_score_pass"})

In [128]:
math_passing_combined_df = passing_combined_df[(passing_combined_df["math_score_pass"] >= 70)]
reading_passing_combined_df = passing_combined_df[(passing_combined_df["reading_score_pass"] >= 70)]
overall_passing_combined_df = passing_combined_df[(passing_combined_df["math_score_pass"] >= 70)\
                                                  &(passing_combined_df["reading_score_pass"] >= 70)]

In [129]:
per_school_count_pass_math = math_passing_combined_df.groupby(["school_name"]).count()["math_score_pass"]
per_school_count_pass_reading = reading_passing_combined_df.groupby(["school_name"]).count()["reading_score_pass"]
per_school_count_overall_pass = overall_passing_combined_df.groupby(["school_name"]).count()["math_score_pass"]

In [130]:
# % passing math (the percentage of students who passed math)
school_summary_df["% Passing Math"] = per_school_count_pass_math/ school_summary_df["Total Students"]
# % passing reading (the percentage of students who passed reading)
school_summary_df["% Passing Reading"] = per_school_count_pass_reading/ school_summary_df["Total Students"]

In [245]:
# % overall passing (the percentage of students who passed math AND reading)
school_summary_df["% Overall passing"] = per_school_count_overall_pass/ school_summary_df["Total Students"]

# formatting
school_summary_df["% Passing Math"] = school_summary_df["% Passing Math"] *100
school_summary_df["% Passing Reading"] = school_summary_df["% Passing Reading"]*100
school_summary_df["% Overall passing"] = school_summary_df["% Overall passing"]*100
per_school_summary = school_summary_df
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall passing
school_name,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,6568.392184,8131.642098,53.513884
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,6598.847067,8073.923364,53.204476
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,9386.712095,9585.462805,89.892107
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,6675.296656,8086.299892,53.527508
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,9339.237057,9713.896458,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,9386.771792,9653.964082,90.582567
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,9413.347686,9703.982777,91.334769
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,6668.006431,8193.327974,54.642283
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,9250.585480,9625.292740,89.227166


## Highest-Performing Schools (by % Overall Passing)

In [132]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = school_summary_df.sort_values(by=["% Overall passing"], ascending = False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Lowest-Performing Schools (by % Overall Passing)

In [133]:
# Sort the schools by % Overall Passing in ascending order and display the top 5 rows.
# Save the results in a DataFrame called "bottom_schools".
bottom_schools = school_summary_df.sort_values(by=["% Overall passing"], ascending = True)
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [134]:
#Perform the necessary calculations to create a DataFrame that lists the average math score 
# \for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [135]:
# sorting out each grade
ninth = combined_df[(combined_df["grade"] == "9th")]
tenth = combined_df[(combined_df["grade"] == "10th")]
eleventh = combined_df[(combined_df["grade"] == "11th")]
twelth = combined_df[(combined_df["grade"] == "12th")]

In [136]:
# getting scores for each grade
ninth = ninth[["school_name","math_score"]]
ninth = ninth.rename(columns = {"math_score":"9th"})

tenth = tenth[["school_name","math_score"]]
tenth = tenth.rename(columns = {"math_score":"10th"})

eleventh = eleventh[["school_name","math_score"]]
eleventh = eleventh.rename(columns = {"math_score":"11th"})

twelth = twelth[["school_name","math_score"]]
twelth = twelth.rename(columns = {"math_score":"12th"})



In [137]:
ninth_average_math = ninth.groupby(["school_name"]).mean()["9th"]
tenth_average_math = tenth.groupby(["school_name"]).mean()["10th"]
eleventh_average_math = eleventh.groupby(["school_name"]).mean()["11th"]
twelth_average_math = twelth.groupby(["school_name"]).mean()["12th"]

ninth_average_math = ninth_average_math.to_frame()
tenth_average_math = tenth_average_math.to_frame()
eleventh_average_math = eleventh_average_math.to_frame()
twelth_average_math = twelth_average_math.to_frame()

In [138]:
math_scores_by_grade = ninth_average_math
math_scores_by_grade ["10th"] = tenth_average_math
math_scores_by_grade["11th"] = eleventh_average_math
math_scores_by_grade["12th"] = twelth_average_math
math_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Scores by Grade

In [139]:
# Create a DataFrame that lists the average reading score for students of each grade level 
# (9th, 10th, 11th, 12th) at each school.

In [140]:
# sorting out each grade
ninth = combined_df[(combined_df["grade"] == "9th")]
tenth = combined_df[(combined_df["grade"] == "10th")]
eleventh = combined_df[(combined_df["grade"] == "11th")]
twelth = combined_df[(combined_df["grade"] == "12th")]

In [141]:
# getting scores for each grade
ninth_reading = ninth[["school_name","reading_score"]]
ninth_reading = ninth_reading.rename(columns = {"reading_score":"9th"})

tenth_reading = tenth[["school_name","reading_score"]]
tenth_reading = tenth_reading.rename(columns = {"reading_score":"10th"})

eleventh_reading = eleventh[["school_name","reading_score"]]
eleventh_reading = eleventh_reading.rename(columns = {"reading_score":"11th"})

twelth_reading = twelth[["school_name","reading_score"]]
twelth_reading = twelth_reading.rename(columns = {"reading_score":"12th"})




In [142]:
ninth_average_reading = ninth_reading.groupby(["school_name"]).mean()["9th"]
tenth_average_reading = tenth_reading.groupby(["school_name"]).mean()["10th"]
eleventh_average_reading = eleventh_reading.groupby(["school_name"]).mean()["11th"]
twelth_average_reading = twelth_reading.groupby(["school_name"]).mean()["12th"]

ninth_average_reading = ninth_average_reading.to_frame()
tenth_average_reading = tenth_average_reading.to_frame()
eleventh_average_reading = eleventh_average_reading.to_frame()
twelth_average_reading = twelth_average_reading.to_frame()

In [143]:
reading_scores_by_grade = ninth_average_reading
reading_scores_by_grade ["10th"] = tenth_average_reading
reading_scores_by_grade["11th"] = eleventh_average_reading
reading_scores_by_grade["12th"] = twelth_average_reading
reading_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [144]:
# Create a table that breaks down school performance based on average spending ranges (per student).
# Use the code provided below to create four bins with reasonable cutoff values to group school spending.

In [177]:
spending_bins = [0, 585, 630, 645, 680]
labels = ["<585", "$585-630", "$630-645", "$645-680"] #

In [178]:
school_spending_df = school_summary_df.copy()

school_spending_df["Per Student Budget"]= school_spending_df["Per Student Budget"].str.replace('$', '')
school_spending_df["Per Student Budget"]=school_spending_df["Per Student Budget"].astype('float64')

/var/folders/gm/l91q8qv12cl1n1jwmtbrztc40000gn/T/ipykernel_29554/2670986679.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  school_spending_df["Per Student Budget"]= school_spending_df["Per Student Budget"].str.replace('$', '')


In [183]:
# Use pd.cut to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], \
                                                             spending_bins, labels =labels, include_lowest=True)
school_spending_df["Per Student Budget"]=school_spending_df["Per Student Budget"].map("${:,.2f}".format)
school_spending_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall passing,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,89.892107,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630


In [209]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall passing"]

In [198]:
# Use the scores above to create a DataFrame called spending_summary.
spending_summary = spending_math_scores.to_frame()
spending_summary["Average Reading Score"] = spending_reading_scores
spending_summary["% Passing Math"] = spending_passing_math
spending_summary["% Passing Reading"] = spending_passing_reading
spending_summary["% Overall passing"] = overall_passing_spending

In [199]:
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall passing
Spending Ranges (Per Student),,,,,
<585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [221]:
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [227]:
# bin the per_school_summary
size_summary_df = school_summary_df.copy()
# Use pd.cut on the "Total Students" column of the per_school_summary DataFrame.
pd.cut(size_summary_df["Total Students"], size_bins, labels =labels, include_lowest=True)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall passing
school_name,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,89.892107
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166


In [233]:
# Create a DataFrame called size_summary that breaks down school performance based on school size (small, medium, or large).
size_summary_df["School Size"] = pd.cut(size_summary_df["Total Students"], size_bins, labels =labels, include_lowest=True)
size_summary = size_summary_df.groupby(["School Size"])

In [235]:
#  Calculate averages for the desired columns. 
size_math_scores = size_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = size_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = size_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = size_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
overall_passing_size = size_summary_df.groupby(["School Size"]).mean()["% Overall passing"]

In [238]:
size_summary = size_math_scores.to_frame()
size_summary["Average Reading Score"] = size_reading_scores
size_summary["% Passing Math"] = size_passing_math
size_summary["% Passing Reading"] = size_passing_reading
size_summary["% Overall passing"] = overall_passing_size
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [241]:
# Use the per_school_summary DataFrame from the previous step to create a new DataFrame called type_summary.

# I USED A DIFFERENT NAME ABOVE.... 
# INSTEAD OF per_school_summary, IT'S size_summary_df ... 
# HOPE IT'S OK!

type_math_scores = size_summary_df.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = size_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = size_summary_df.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = size_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]
overall_passing_type = size_summary_df.groupby(["School Type"]).mean()["% Overall passing"]


In [244]:
type_summary = type_math_scores.to_frame()
type_summary["Average Reading Score"] = type_reading_scores
type_summary["% Passing Math"] = type_passing_math
type_summary["% Passing Reading"] = type_passing_reading
type_summary["% Overall passing"] = overall_passing_type
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
